In [1]:
import os
import sqlite3

import pandas as pd
from alice_lri import Intrinsics
from jaraco.functools import assign_params

from scripts.common.helper.entities import IntrinsicsFrameResult, DatasetFrameGt, DatasetFrameScanlineGt, DatasetEntity
from scripts.local.runtime import measure_rtst_times
from scripts.local.paper.helper.utils import df_to_latex, write_paper_data, df_from_sql_table, compute_metrics

from scripts.common.helper.entities import DatasetFrame
from scripts.common.load_env import load_env
load_env()
DB_PATH = os.getenv("LOCAL_SQLITE_MASTER_DB")
robust_point_count_threshold = 64
experiment_id = 38

In [2]:
conn = sqlite3.connect(DB_PATH)
datasets_df = df_from_sql_table(conn, DatasetEntity.__table__)
dataset_frames_df = df_from_sql_table(conn, DatasetFrame.__table__)
frame_results_df = df_from_sql_table(conn, IntrinsicsFrameResult.__table__, "experiment_id == ?", (experiment_id,))
# frame_gt_df = df_from_sql_table(conn, DatasetFrameGt.__table__)
frame_gt_df = df_from_sql_table(conn, "dataset_frame_empirical")
# scanline_gt_df = df_from_sql_table(conn, DatasetFrameScanlineGt.__table__)
scanline_gt_df = df_from_sql_table(conn, "dataset_frame_scanline_info_empirical")

In [6]:
non_robust_frame_ids = scanline_gt_df[scanline_gt_df["points_count"] < robust_point_count_threshold]["dataset_frame_id"].unique()

frame_full_df = pd.merge(dataset_frames_df, frame_results_df, left_on="id", right_on="dataset_frame_id", how="inner")
frame_full_df = pd.merge(datasets_df, frame_full_df, left_on="id", right_on="dataset_id", how="inner")
frame_full_df = pd.merge(frame_full_df, frame_gt_df, on="dataset_frame_id", how="inner", suffixes=("_estimated", "_gt"))
frame_full_df["robust"] = True
frame_full_df.loc[frame_full_df["dataset_frame_id"].isin(non_robust_frame_ids), "robust"] = False

In [25]:
scanline_metrics_all_df = frame_full_df.groupby(["name"])\
    .apply(lambda df: compute_metrics(df, "scanlines_count_gt", "scanlines_count_estimated"), include_groups=False)\
    .reset_index()\
    .assign(subset="All")

scanline_metrics_robust_only_df = frame_full_df[frame_full_df["robust"]].groupby(["name"]) \
    .apply(lambda df: compute_metrics(df, "scanlines_count_gt", "scanlines_count_estimated"), include_groups=False) \
    .reset_index()\
    .assign(subset="n(l) >= 64")

scanline_metrics_df = pd.concat([scanline_metrics_all_df, scanline_metrics_robust_only_df], axis=0)
scanline_metrics_df = scanline_metrics_df.sort_values(by=["name", "subset"], ascending=[False, True])

In [26]:
scanline_metrics_df

,name,samples,incorrect,oa,mp,mr,mf1,wp,wr,wf1,subset
1,kitti,47885.0,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,All
1,kitti,47543.0,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,n(l) >= 64
0,durlar,145916.0,130.0,99.910908,92.977032,93.668454,93.307977,99.920694,99.910908,99.915145,All
0,durlar,130757.0,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,n(l) >= 64
